# Useful functions
Some functions that might not be obvious but are helpful to interact with files and such. In general it's good to load these libraries to get started.

In [2]:
import numpy as np
import random
import hashlib

## Read a file to get the bitstream (1s and 0s)

In [17]:
def get_raw_bits(file):
    # Read a file and return an array of bits
    dat = np.fromfile(file, dtype='uint8')
    return list(np.unpackbits(dat))

## Check a file is really the same
Some file formats, such as jpeg or png, have built-in error correction so sometimes visual inspection might not reveal errors that happened during decoding. This function compares two files and returns a boolean answer.

In [19]:
def compare_files(file1, file2):
    # Compare file1 and file2 using SHA256 checksum, if they are the same return True else return False
    with open(file1, 'rb') as fi:
        data1 = fi.read()
    hash1 = hashlib.sha256(data1).hexdigest()
    with open(file2, 'rb') as fi:
        data2 = fi.read()
    hash2 = hashlib.sha256(data2).hexdigest()
    return hash1 == hash2

## Filling out arrays
Two functions that can be used to pad an array out at the beginning or end.

In [22]:
def pad_b_arr(array, size, padding=0):
    # Pads out array to size with padding (default 0) at beginning
    while len(array)<size:
        array.insert(0, 0)
    return array

def pad_e_arr(array, size, padding=0):
    # Pads out array to size with padding (default 0) at end
    while len(array)<size:
        array.insert(len(array), padding)
    return array

# Simulating synthesis, PCR and sequencing

In [23]:
def sim_seq_simple(sequences, dropout = 0.01, insert_rate = 0.00042, del_rate = 0.00188, sub_rate = 0.00407):
    # Takes sequences as a list of unique DNA sequences and simulates: synthesis, PCR and NGS
    # Dropout is the dropout rate (i.e. the number of sequences that are not recovered)
    # For baselevel errors we have insert_rate for insertions, del_rate for deletions and sub_rate for substitutions.
    # Default values for base errors are taken from https://www.nature.com/articles/nbt.4079
    s = sequences.copy()
    random.shuffle(s)
    for _ in range(0, int(dropout*len(s))):
        s.pop()
    
    for i in range(len(s)):
        seq_l = list(s[i])
        mod = 0 # Inserting messes with indexing so we skip over inserted bases using this counter
        for base_i in range(len(seq_l)):
            mutate_rand = random.random()
            if mutate_rand < insert_rate:
                # Insert random nucleotide after this base
                seq_l.insert(base_i+mod, random.choice(['A','C', 'T', 'G']))
                mod+=1
            elif mutate_rand > insert_rate and mutate_rand < (del_rate+insert_rate):
                # Delete this nucleotide (later)
                seq_l[base_i+mod] = '_'
            elif mutate_rand > (insert_rate+del_rate) and mutate_rand < (del_rate+insert_rate+sub_rate):
                # Substitute base
                seq_l[base_i+mod] = random.choice(['A', 'C', 'T', 'G'])
        if '_' in seq_l:
            for dels in range(seq_l.count('_')):
                seq_l.remove('_')
        s[i] = ''.join(seq_l)
    return s

In [24]:
def binorm_sim(n, p=0.95, cyc=10):
    # simulate the number of molecules after "c" cycle
    # n: number of template molecules
    # p: probability of success PCR
    for i in range(cyc):
        n += np.random.binomial(n, p, 1)
    return n.item(0)

def mutate(seq, insert_rate = 0.00008, del_rate = 0.00144, sub_rate = 0.00298):
    # Mutate a single sequence based on previously observed error rates
    seq_l = list(seq)
    mod = 0 # Inserting messes with indexing so we skip over inserted bases using this counter
    for base_i in range(len(seq_l)):
        mutate_rand = random.random()
        if mutate_rand < insert_rate:
            # Insert random nucleotide after this base
            seq_l.insert(base_i+mod, random.choice(['A','C', 'T', 'G']))
            mod+=1
        elif mutate_rand > insert_rate and mutate_rand < (del_rate+insert_rate):
            # Delete this nucleotide (later)
            seq_l[base_i+mod] = '_'
        elif mutate_rand > (insert_rate+del_rate) and mutate_rand < (del_rate+insert_rate+sub_rate):
            # Substitute base
            seq_l[base_i+mod] = random.choice(['A', 'C', 'T', 'G'])
    if '_' in seq_l:
        for dels in range(seq_l.count('_')):
            seq_l.remove('_')
    return ''.join(seq_l)

def sim_full(seqs, dilution=8, cov=10, p_PCR=0.95, cyc_PCR=18, error_rates=[0.00008, 0.00144, 0.00298]):
    # Simulates a full experiment from synthesis, PCR to sequencing
    # This is based on the model in fig 5b of https://www.nature.com/articles/s41467-020-16958-3
    # INPUT: an array of all sequences that should be ordered to write a file to DNA
    # OUTPUT: an array of sequences that can be expected to be returned from a typical experiment
    #
    # PARAMETERS
    # dilution: how many molecules per sequences are present in a diluted pool on average (default 8)
    # cov: average coverage during sequencing, ie how many times a sequence is read (default 18)
    # p_PCR: chance of succesfull replication during PCR (default 0.95)
    # cyc_PCR: how many PCR cycles are used for amplification (default 10)
    # error_rates: array for error rates per base [insertions, deletions, substitutions] (default [0.00008, 0.00144, 0.00298])
    
    n_seqs = len(seqs)
    indices = list(range(n_seqs)) # UMI for each sequence
    
    ## Simulate synthesis on an array and dilution down to 8 average copies per sequence
    mol_counts = np.random.normal(loc=10**3, scale=int(3.2e2), size=n_seqs) # How many copies after synthesis (scaled down)
    copies_Syn = np.concatenate([np.repeat(idx, int(c)) for idx, c in zip(indices, mol_counts)])
    in_sample = np.random.choice(copies_Syn, size=n_seqs*dilution, replace=False)
    counts = np.bincount(in_sample)
    
    ## Simulate the PCR process
    post_PCR = [0 for idx in indices]
    for idx in indices:
        post_PCR[idx] = binorm_sim(counts[idx], p=p_PCR, cyc=cyc_PCR)
    copies_PCR = np.concatenate([np.repeat(idx, int(c)) for idx, c in zip(indices, post_PCR)])
    sequenced_UMI = np.random.choice(copies_PCR, size=n_seqs*cov, replace=False) # UMIs of molecules in sequencing result
    
    ## Sprinkle in some errors
    final_seqs = [0 for _ in range(len(sequenced_UMI))]
    for i, UMI in enumerate(sequenced_UMI):
        final_seqs[i] = mutate(seqs[UMI], error_rates[0], error_rates[1], error_rates[2])
    return final_seqs

In [26]:
sim_full(['CTGAAATAAATAAATAGAGTCTACCCTGATCCATTGCTTTATTTACCGAGTCTTATATACCCCATTGCCTTCCCACTAGAGTCTCACCTGATCCATTGCTTTATTTACCGAGTCTTATCCATACCATTGCCAAACCATTAGAGTCAAACCTTCAGCACTGCCTATCGTTCCGTCCTAACCA', 
          'GCCGTTGTTTGTACTGTGATGCTCTTCCCTGATCCATTGCTTTATTTACCGAGTCTTATCTGATCCATTGCCTAACCCACCGAGTCCTTTCTGATCCATTGCTACCACCTCCGAGTCTAACCTTCAGCATTGCCCAAGGTTCTGAGCGAAATG'])

['CTGAAATAAATAAATAAGAGTCTACCCTGATCCATTGCTTTATTTACCGAGTCTTATATACCCCATTGCCTTCCCACTAGAGTCTCACCTGATCCATTGCTTTATTACCGAGTCTTATCCATACCATTGCCAAACCATTAGAGTCAAACCTTCAGCACTGCCTATCGTTCCGTCCTAACCA',
 'CTGAAATAAATAAATAGAGTCTACCCTGATCCATTGCTTTATTTACCGAGTCTTATATACCCCATTGCCTTCCCACTAGAGTCTCACCTGATCCATTGCTTTATTTACCGAGTCTTATCCATACCATTGCCAAACCATTAGAGTCAAACCTTCAGCACTGCCTATCGTTCCGTCCTAACCA',
 'GCAGTTGTTTGTACTGTGATGCTCTTCCCTGATCCATTGCTTTATTTACCGAGTCTTATCTGATCCATTGCCTAACCCACCGAGTCCTTTCTGATCCATTGCTACCACCTCCGAGTCTAACCTTCAGCATTGCCCAAGGTTCTGAGCGAAATG',
 'CTGAAATAAATAAATAGAGTCTACCCTGATCCATTGCTTTATTTACCGAGTCTTATATACCCCATTGCCTTCCCACTAGAGTCTCACCTGATCCATTGCTTTATTTACCGAGTCTTATCCATACCATTGCCAAACCATTAGAGTCAAACCTTCAGCACTGCCTTTCGTTCCGTCCTAACCA',
 'CTGAAATAAATAAATAGAGTCTACCCTGATCCATTGCTTTATTTACCGAGTCTTATATACCCCATTGCCTTCCCACTAGAGTCTCACCTGATCCATTGCTTTATTTACCGAGTCTTATCCATACCATTGCCAAACCATTAGAGTCAAACCTTCAGCACTGCCTATCGTTCCGTCCTAACCA',
 'GCCGTTGTTTGTACTGTGATGCTCTTCCCTGATCCATTGCTTTATTTACCGAGTCTTATCTGATCCATTGCCTAACCCACCGAGTCCTTTCTGATC